# Overview

The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

<h2>Files available :</h2>

<ul>
<li>sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.</li>
<li>test.csv - the test set. You need to forecast the sales for these shops and products for November 2015. </li>
<li>sample_submission.csv - a sample submission file in the correct format. items.csv - supplemental information about the items/products. </li>
<li>item_categories.csv - supplemental information about the items categories. </li>
<li>shops.csv- supplemental information about the shops.</li>
 <ul>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:20,.2f}'.format
plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 200

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor



from sklearn.metrics import mean_squared_error

In [ ]:
import pickle

In [ ]:
import xgboost as xgb

# Understand the problem

In [ ]:
train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")


items = pd.read_csv("../input/predict-future-sales-translated-dataset/items_en.csv")
shops = pd.read_csv("../input/predict-future-sales-translated-dataset/shops_en.csv")
categories = pd.read_csv("../input/predict-future-sales-translated-dataset/item_categories_en.csv")

submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
print(' train set  :{} \n test set : {}'.format(list(train.columns), list(test.columns)))

In [ ]:
print(' shape train {} \n shape test {} \n total data {}'.format(train.shape, test.shape, train.shape[0]+test.shape[0]))
print(" train dataset have null: {} \n test dataset have null: {} ".format(train.isnull().sum().any(), test.isnull().sum().any()))


#### Review of same items in train and test set

Here we will can see that the values in the item shop and the shop id not are same, therefore later we going to remove the items that not make match between dataset, it is because is not necesary.

In [ ]:
#review shop_id in train set and test set
print(' unique values of shop_id in train set: \n {} \n\n unique values of shop_id in test set: \n {}'.format(len(train.shop_id.unique()),len(test.shop_id.unique())))

#### How are the frequencies in the dates

In [ ]:
# how are items most freqs 
datesfreq = train.groupby('date_block_num')['shop_id'].count().sort_values(ascending=False).head()

In [ ]:
dates = train[train.date_block_num.isin(list(datesfreq.index))]
dates = dates.groupby('date')['shop_id'].count().sort_values(ascending=True).reset_index()
dates = dates.set_index('date')
dates.index = pd.to_datetime(dates.index)
dates.columns = ['freq']

In [ ]:
print('months: {} \n\n days: {}'.format(sorted(list(dates.index.month.unique())) ,sorted(list(dates.index.day.unique()))))

In the following figure it is possible to see that the peak value was in December, it's is logical because is an age when there is a lot of movements transaction for christmas, even the percentil 0.95 is alocated between November and January. 

In [ ]:
maxvalue = dates.sort_values(by='freq',ascending=False)[:1]
ax = dates.plot(color='orange')
ax.set_xlabel('Date')
ax.set_ylabel('Freq')
ax.axvline(maxvalue.index, color='green', linestyle='--', alpha=0.3)
ax.axhline(maxvalue.values, color='blue', linestyle='--', alpha=0.3)
ax.axhline(dates.quantile(0.95).values, color='purple', linestyle='--', alpha=0.3)
ax.text(maxvalue.index,maxvalue.values, '  maxvalue : {} : {}'.format(str(maxvalue.index.date[0]),str(maxvalue.values[0][0])), fontsize=6)
ax.text(maxvalue.index,dates.quantile(0.95).values, ' percentile 0.95', fontsize=6)
plt.show()

In [ ]:
print('How many item_id are? : {}'.format(len(train.item_id.unique())))

In [ ]:
train['shop_id'].value_counts(normalize=True).plot(kind='bar', alpha=0.7, figsize=(12,3))
plt.title("Shop Id Values")

In [ ]:
train['item_price'].plot(kind='hist', alpha=0.7, color='orange', figsize=(12,3))
plt.title("Item price Histogram")

In [ ]:
train['item_cnt_day'].plot(kind='hist', alpha=0.7, color='green', figsize=(12,3))
plt.title('Item count by day Histogram')

In [ ]:
train['item_cnt_day'].sort_values(ascending=False)[:5]

In [ ]:
train[train['item_cnt_day'] == 2169]

Item 11373 was sold 2169 times at shop 12 on a single day in Octuber.

In [ ]:
items[items['item_id'] == 11373]

In [ ]:
train[train['item_id'] == 11373].head(5)

In [ ]:
train[train['item_id']== 11373]['item_cnt_day'].median()

In [ ]:
train[train['item_id']== 11373]['item_cnt_day'].plot(kind='hist',figsize=(12,3))
plt.text(1000,300,'median item_cnt_day: {}'.format(train[train['item_id']== 11373]['item_cnt_day'].median()))

Now  it is possible to see that value 2169 in column item_cnt_day is a value anomaly, so we going to impute this value

In [ ]:
train = train[train['item_cnt_day'] < 2000]
train['item_price'].sort_values(ascending=False)[:5]

##### Price 307.980

In [ ]:
train[train['item_price'] == 307980]

In [ ]:
items[items['item_id'] == 6066]

In [ ]:
train[train['item_id']== 6066]

Radmin 3 to 522 people, it mean thata all price is by 522 people.
It is only item, there is not point benchmark to understand the value, so it will be drop from training set. 

In [ ]:
train = train[train['item_price'] < 300000]

##### Price -1 

In [ ]:
train['item_price'].sort_values()[:5]

In [ ]:
train[train['item_price'] == -1]

In [ ]:
items[items['item_id'] == 2973]

In [ ]:
train[train['item_id'] == 2973].head(5)

##### Stores
We going to how much store there are and if they are duplicated

In [ ]:
len(train['shop_id'].unique())

In [ ]:
len(test['shop_id'].unique())

In [ ]:
shops.T

Effectively store are duplicated, so we going to replace id with a id most representative

In [ ]:
train.loc[train['shop_id'] == 0, 'shop_id'] = 57
test.loc[test['shop_id'] == 0, 'shop_id'] = 57

train.loc[train['shop_id'] == 1, 'shop_id'] = 58
test.loc[test['shop_id'] == 1, 'shop_id'] = 58

train.loc[train['shop_id'] == 10, 'shop_id'] = 11
test.loc[test['shop_id'] == 10, 'shop_id'] = 11

Also there are store that have as name, the name city, so it could be complicated for separed the same store, so we going to remove name city from store.

In [ ]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda row: row[0])

In [ ]:
shops[shops['city'] == '!']

In [ ]:
shops['city'] = shops['city'].str.replace('!','Yakutsk')

In [ ]:
shops['city'].unique()

In [ ]:
from sklearn import preprocessing

##### Encode Name store 

Now we going to encode name store to make modelling most easier for processing. 

In [ ]:
le = preprocessing.LabelEncoder()
le.fit_transform(shops['city'])

In [ ]:
shops['city_label'] = le.fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis=1, inplace=True)
shops.head()

### Items_Analysis

In [ ]:
items_train = train['item_id'].nunique()
items_test = test['item_id'].nunique()
print('items_train {} \n item_test {}'.format(items_train, items_test))

In [ ]:
items_train_list = list(train['item_id'].unique())
items_test_list = list(test['item_id'].unique())

flag = 0
if(set(items_test_list).issubset(set(items_train_list))):
    flag=1
if (flag) : 
    print ("Yes, list is subset of other.") 
else : 
    print ("No, list is not subset of other.") 

It mean that test df there is value that is not on train set

In [ ]:
len(set(items_test_list).difference(items_train_list))

In [ ]:
categories_in_test = items.loc[items['item_id'].isin(sorted(test['item_id'].unique()))].item_category_id.unique()

In [ ]:
categories.loc[~categories['item_category_id'].isin(categories_in_test)].T

In [ ]:
le = preprocessing.LabelEncoder()
main_categories = categories['item_category_name'].str.split('-')
categories['main_category_id'] = main_categories.map(lambda row: row[0].strip())
categories['main_category_id'] = le.fit_transform(categories['main_category_id'])
categories['sub_category_id'] = main_categories.map(lambda row: row[1].strip() if len(row) > 1 else row[0].strip())
categories['sub_category_id'] = le.fit_transform(categories['sub_category_id'])

In [ ]:
categories.head()

In [ ]:
from itertools import product

Testing generation of cartesian product for the month of February in 2013

In [ ]:
shops_in_jan = train.loc[train['date_block_num'] == 0,'shop_id'].unique()
items_in_jan = train.loc[train['date_block_num']==0, 'item_id'].unique()

In [ ]:
jan = list(product(*[shops_in_jan, items_in_jan,[0]]))

In [ ]:
shops_in_feb = train.loc[train['date_block_num']==1, 'shop_id'].unique()
items_in_feb = train.loc[train['date_block_num']==1, 'item_id'].unique()
feb = list(product(*[shops_in_feb, items_in_feb, [1]]))

In [ ]:
cartesian_test = []
cartesian_test.append(np.array(jan))
cartesian_test.append(np.array(feb))

In [ ]:
cartesian_test

In [ ]:
cartesian_test = np.vstack(cartesian_test)
cartesian_test

In [ ]:
cartesian_test_df = pd.DataFrame(cartesian_test, columns = ['shop_id', 'item_id', 'date_block_num'])

In [ ]:
months = train['date_block_num'].unique()

In [ ]:
from tqdm import tqdm_notebook

def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    
    return df

In [ ]:
months = train['date_block_num'].unique()

In [ ]:
cartesian = []
for month in months:
    shops_in_month = train.loc[train['date_block_num']==month, 'shop_id'].unique()
    items_in_month = train.loc[train['date_block_num']==month, 'item_id'].unique()
    cartesian.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))

In [ ]:
cartesian_df = pd.DataFrame(np.vstack(cartesian), columns = ['shop_id', 'item_id', 'date_block_num'], dtype=np.int32)

In [ ]:
x = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().rename('item_cnt_month').reset_index()
x.head()

In [ ]:
new_train = pd.merge(cartesian_df, x, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)

In [ ]:
new_train['item_cnt_month'] = np.clip(new_train['item_cnt_month'], 0, 20)

In [ ]:
del x
del cartesian_df
del cartesian
del cartesian_test
del cartesian_test_df
del feb
del jan
del items_test_list
del items_train_list
#del train

In [ ]:
new_train.sort_values(['date_block_num','shop_id','item_id'], inplace = True)
new_train.head()

In [ ]:
test.insert(loc=3, column='date_block_num', value=34)

In [ ]:
test['item_cnt_month'] = 0

In [ ]:
new_train = new_train.append(test.drop('ID', axis = 1))

In [ ]:
new_train = pd.merge(new_train, shops, on=['shop_id'], how='left')
new_train.head()

In [ ]:
new_train = pd.merge(new_train, items.drop('item_name', axis = 1), on=['item_id'], how='left')
new_train.head()

In [ ]:
new_train = pd.merge(new_train, categories.drop('item_category_name', axis = 1), on=['item_category_id'], how='left')
new_train.head()

#### Lag datasets

Now, in this step we going to generate lag over each column

In [ ]:
def generate_lag(train, months, lag_column):
    for month in months:
        # Speed up by grabbing only the useful bits
        train_shift = train[['date_block_num', 'shop_id', 'item_id', lag_column]].copy()
        train_shift.columns = ['date_block_num', 'shop_id', 'item_id', lag_column+'_lag_'+ str(month)]
        train_shift['date_block_num'] += month
        train = pd.merge(train, train_shift, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return train

In [ ]:
del items
del categories
del shops
del test

In [ ]:
new_train = downcast_dtypes(new_train)

In [ ]:
import gc
gc.collect()

In [ ]:
%%time
new_train = generate_lag(new_train, [1,2,3,4,5,6,12], 'item_cnt_month')

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'item_id'])['item_cnt_month'].mean().rename('item_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'item_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'item_month_mean')
new_train.drop(['item_month_mean'], axis=1, inplace=True)

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'shop_id'])['item_cnt_month'].mean().rename('shop_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'shop_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'shop_month_mean')
new_train.drop(['shop_month_mean'], axis=1, inplace=True)

In [ ]:
new_train.columns

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_month'].mean().rename('shop_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'shop_id', 'item_category_id'], how='left')
new_train = generate_lag(new_train, [1, 2], 'shop_category_month_mean')
new_train.drop(['shop_category_month_mean'], axis=1, inplace=True)

In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'main_category_id'])['item_cnt_month'].mean().rename('main_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'main_category_id'], how='left')

new_train = generate_lag(new_train, [1], 'main_category_month_mean')
new_train.drop(['main_category_month_mean'], axis=1, inplace=True)



In [ ]:
%%time
group = new_train.groupby(['date_block_num', 'sub_category_id'])['item_cnt_month'].mean().rename('sub_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['date_block_num', 'sub_category_id'], how='left')

new_train = generate_lag(new_train, [1], 'sub_category_month_mean')
new_train.drop(['sub_category_month_mean'], axis=1, inplace=True)

In [ ]:
new_train.head()

In [ ]:
new_train['month'] = new_train['date_block_num'] % 12

Add Holiday days to ddataset

In [ ]:
holiday_dict = {
    0: 6,
    1: 3,
    2: 2,
    3: 8,
    4: 3,
    5: 3,
    6: 2,
    7: 8,
    8: 4,
    9: 8,
    10: 5,
    11: 4,
}

In [ ]:
new_train['holidays_in_month'] = new_train['month'].map(holiday_dict)

In [ ]:
moex = {
    12: 659, 13: 640, 14: 1231,
    15: 881, 16: 764, 17: 663,
    18: 743, 19: 627, 20: 692,
    21: 736, 22: 680, 23: 1092,
    24: 657, 25: 863, 26: 720,
    27: 819, 28: 574, 29: 568,
    30: 633, 31: 658, 32: 611,
    33: 770, 34: 723,
}

In [ ]:
new_train['moex_value'] = new_train.date_block_num.map(moex)

In [ ]:
new_train = downcast_dtypes(new_train)


In [ ]:
import xgboost as xgb

In [ ]:
new_train = new_train[new_train.date_block_num > 11]

In [ ]:
import gc
gc.collect()

In [ ]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            df[col].fillna(0, inplace=True)         
    return df

new_train = fill_na(new_train)

In [ ]:
def xgtrain():
    regressor = xgb.XGBRegressor(n_estimators = 5000,
                                 learning_rate = 0.01,
                                 max_depth = 10,
                                 subsample = 0.5,
                                 colsample_bytree = 0.5)
    
    regressor_ = regressor.fit(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                               new_train[new_train.date_block_num < 33]['item_cnt_month'].values, 
                               eval_metric = 'rmse', 
                               eval_set = [(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num < 33]['item_cnt_month'].values), 
                                           (new_train[new_train.date_block_num == 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num == 33]['item_cnt_month'].values)
                                          ], 
                               verbose=True,
                               early_stopping_rounds = 50,
                              )
    return regressor_

In [ ]:
%%time
regressor_ = xgtrain()

In [ ]:
predictions = regressor_.predict(new_train[new_train.date_block_num == 34].drop(['item_cnt_month'], axis = 1).values)

In [ ]:
regressor_.save_model("model.json")

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27

cols = new_train.drop('item_cnt_month', axis = 1).columns
plt.barh(cols, regressor_.feature_importances_)
plt.show()

In [ ]:
submission['item_cnt_month'] = predictions

In [ ]:
submission.to_csv('sales_faster_learn.csv', index=False)

In [ ]:
sub_df.to_csv('Submission_3.csv',index=False)